In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [3]:
def rank3tensor_decomposition_ALS(matrix, k, alpha=0.01, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
    costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
    costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))
    
    trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
    trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
    trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)

    for _ in range(nbiter):
        sess.run(trainA, feed_dict={X: matrix})
        sess.run(trainB, feed_dict={X: matrix})
        sess.run(trainC, feed_dict={X: matrix})

    return sess.run((A, B, C), feed_dict={X: matrix})

In [4]:
def rank3tensor_decomposition_ALS_closed(matrix, k, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    for _ in range(nbiter):
        A = tf.matmul(tf.matmul(X0, khatrirao_product(C, B)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(B), B)))
        B = tf.matmul(tf.matmul(X1, khatrirao_product(C, A)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(A), A)))
        C = tf.matmul(tf.matmul(X2, khatrirao_product(B, A)), 
                      tf.multiply(tf.matmul(tf.transpose(B), B), tf.matmul(tf.transpose(A), A)))
        
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)
    
    return sess.run((A, B, C), feed_dict={X: matrix})

# Test: Method 1

In [18]:
# x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
# x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
# x = np.zeros((3, 4, 2))
# x[:, :, 0] = x1
# x[:, :, 1] = x2
x1 = np.array([[1, 0], [0, 1]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0]], dtype=np.float32)
x = np.zeros((2, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [25]:
A, B, C = rank3tensor_decomposition_ALS(x, k=2, nbiter=5000)

In [26]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [27]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(2))

In [28]:
mat

array([[[ 1.01805404e+00,  7.60583447e-04],
        [ 9.47989523e-03, -5.19108173e-04]],

       [[ 1.36210471e-02, -5.81914122e-04],
        [ 9.80289966e-01, -2.32521363e-03]]])

In [29]:
x

array([[[ 1.,  0.],
        [ 0., -1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

# Method 2

In [30]:
A, B, C = rank3tensor_decomposition_ALS_closed(x, k=5, nbiter=1)

In [31]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [32]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [33]:
mat

array([[[2.70355315e+21, 8.37888282e+20],
        [7.44855732e+22, 2.30846531e+22]],

       [[7.71701398e+21, 2.39166573e+21],
        [2.12733711e+23, 6.59306752e+22]]])

In [34]:
x

array([[[ 1.,  0.],
        [ 0., -1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

In [35]:
A, B, C

(array([[-0.39003256,  0.6316668 , -1.2086369 ,  5.3018904 , -0.37994573],
        [ 2.8968601 ,  3.1896248 , -4.087788  , 15.135282  , -1.0436755 ]],
       dtype=float32),
 array([[ 1.3073799e+02, -2.3024108e+00, -1.6739840e+01, -1.5776406e+03,
         -7.3333824e+01],
        [-1.5086838e+03,  2.7211609e+03, -7.6572314e+03, -4.3386832e+04,
         -1.4674852e+03]], dtype=float32),
 array([[-1.81740212e+15,  4.06312281e+15,  1.51230705e+16,
         -3.23168939e+17,  7.57342522e+14],
        [-5.63239998e+14,  1.25924052e+15,  4.68694910e+15,
         -1.00156885e+17,  2.34716557e+14]], dtype=float32))